In [1]:
import pandas as pd


In [3]:
df = pd.read_excel('User_record.xlsx')
df

,name,contact,Age,Gender,Address,Email,DOB,Contacting web URL,Disease,Blood,Hobby,Occupation,Education,Marital,Sleeping,Income,Internet,Stress,Savings
0,Melissa Adams,58755984418,43.0,NaN,"8541 Michael Fork Suite 850\nSouth Richard, MS...",jose86@myers.com,1988-08-16 00:00:00,https://barker-webb.org/,Asthma,AB+,Gardening,Engineer,High School,Divorced,6.0,183291.0,1.0,High,690345.0
1,Sara Gilbert,26841512445,53.0,NaN,"867 Fields Mills Apt. 747\nNew Richardchester,...",cody75@yahoo.com,1957-03-08 00:00:00,https://www.shepard.biz/,Diabetes,AB+,Sports,Artist,PhD,Single,6.0,130022.0,NaN,Moderate,NaN
2,Kenneth Irwin,37594086955,NaN,Female,"4748 Mark Bypass Suite 779\nMichellechester, M...",christianwheeler@gmail.com,NaN,http://www.johnson.com/,Diabetes,A-,Sports,Doctor,Master's,Divorced,7.0,86283.0,7.0,High,NaN
3,Jacqueline White,97044307674,NaN,Male,USNV Williams\nFPO AP 97072,shannonroach@hotmail.com,2006-12-13 00:00:00,http://www.flores.com/,Allergies,AB+,Music,Engineer,Bachelor's,Married,9.0,71820.0,NaN,High,450353.0
4,Steven Johnson,77521399388,35.0,Female,"56554 Martin Fork\nEast Kristinport, IA 08731",NaN,1980-06-25 00:00:00,https://www.mills.com/,Asthma,B-,Cooking,Designer,High School,Divorced,6.0,22640.0,NaN,Low,294632.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,Brooke Cox,14160788606,49.0,Female,8901 Christopher Villages Apt. 314\nNew Michae...,kirbychad@yahoo.com,1971-11-21 00:00:00,http://hale.net/,Allergies,AB+,Reading,NaN,Master's,Married,8.0,174485.0,7.0,Low,956248.0
5996,Manuel Zuniga IV,72217659959,46.0,Female,"4017 Gary Springs\nNoahchester, CO 64090",sarahjenkins@gmail.com,1966-12-16 00:00:00,https://www.taylor-gonzalez.com/,Allergies,O-,Cooking,Doctor,High School,Single,6.0,157134.0,1.0,Low,179563.0
5997,Robert Cooper,36798939766,40.0,Female,"4273 Robertson Fort\nStephensonview, HI 83716",danielletaylor@hotmail.com,1976-08-04 00:00:00,http://www.chen.biz/,NaN,A-,Music,Teacher,PhD,Divorced,9.0,85442.0,NaN,Low,255523.0
5998,Nicole Evans,89800626206,21.0,Male,"43952 James Spurs\nMelissaland, IN 38233",bjenkins@gmail.com,1998-07-27 00:00:00,http://www.page.info/,Heart Disease,A-,Traveling,Teacher,PhD,Single,8.0,84796.0,7.0,Low,NaN
